In [2]:
!cd /u/luchar/data && git clone https://github.com/mattgroh/fitzpatrick17k

Cloning into 'fitzpatrick17k'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 79 (delta 26), reused 36 (delta 11), pack-reused 0
Unpacking objects: 100% (79/79), 4.59 MiB | 1.06 MiB/s, done.
Updating files: 100% (19/19), done.


In [1]:
path_to_csv = 'fitzpatrick17k/fitzpatrick17k.csv'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv(path_to_csv)

In [4]:
df

,md5hash,fitzpatrick_scale,fitzpatrick_centaur,label,nine_partition_label,three_partition_label,qc,url,url_alphanum
0,5e82a45bc5d78bd24ae9202d194423f8,3,3,drug induced pigmentary changes,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicmminoc...
1,fa2911a9b13b6f8af79cb700937cc14f,1,1,photodermatoses,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicpphoto...
2,d2bac3c9e4499032ca8e9b07c7d3bc40,2,3,dermatofibroma,benign dermal,benign,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicdderma...
3,0a94359e7eaacd7178e06b2823777789,1,1,psoriasis,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppsori...
4,a39ec3b1f22c08a421fa20535e037bba,1,1,psoriasis,inflammatory,non-neoplastic,NaN,https://www.dermaamin.com/site/images/clinical...,httpwwwdermaamincomsiteimagesclinicalpicppsori...
...,...,...,...,...,...,...,...,...,...
16572,cd90e491ddaa92f0f4eb07f73aa09f64,5,5,scleroderma,inflammatory,non-neoplastic,NaN,http://atlasdermatologico.com.br/img?imageId=4529,httpwwwatlasdermatologicocombrimgimageId4529.jpg
16573,5f1ed6de6a9110d7dc580a6a0312af63,4,5,pityriasis rosea,inflammatory,non-neoplastic,NaN,http://atlasdermatologico.com.br/img?imageId=5630,httpwwwatlasdermatologicocombrimgimageId5630.jpg
16574,f198aaf1f0550c2464b285454d34926e,3,3,myiasis,inflammatory,non-neoplastic,NaN,http://atlasdermatologico.com.br/img?imageId=4609,httpwwwatlasdermatologicocombrimgimageId4609.jpg
16575,6214de2e915835014235a1839cbc5938,-1,5,vitiligo,inflammatory,non-neoplastic,NaN,http://atlasdermatologico.com.br/img?imageId=7930,httpwwwatlasdermatologicocombrimgimageId7930.jpg


In [18]:
from pathlib import Path
import requests

In [55]:
fitz_dir = Path('../data/fitzpatrick17k/')
image_dir = fitz_dir / 'images'
image_dir.mkdir(exist_ok=True)
missing_file = (fitz_dir / 'missing.txt')
missing_file.touch()

In [103]:
for i, row in df.iterrows():
    filename = row[0]
    image_url = row[7]
    if not isinstance(image_url, str):
        print('bad url', filename, image_url)
        with open(missing_file, 'a') as f:
            f.write(filename + '\n')
        continue
    ext = Path(image_url).suffix
    if (image_dir / f'{filename}{ext}').exists():
        continue
    response = requests.get(image_url, headers={'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})
    if not response.ok:
        print(filename, end='\t')
        print(response)
    else:
        with open(image_dir / f'{filename}{ext}','wb') as f:
            f.write(response.content)

bad url fc63008d9d80b373d2d8f84dfd034446 nan
bad url 480fd80167d7865ce9aafe14aba4ec9d nan
bad url acbc68bd9e35591b8344ab8b5e51f458 nan
bad url 93997108c3ae3beacc108f8829dc2302 nan
bad url 9d994f1278a9788612fa2af179328c31 nan
bad url 6712381a925124e968bcac89481c90e0 nan
bad url bb23fa10c71a57d17a8c7eb4773db237 nan
bad url d653edb6aa702f095a9b00550738a821 nan
bad url 8cfe54501fe00ddc74096b7cd67aa1e1 nan
bad url 85e11dd1c6f60d02f16516fdfac9c7f7 nan
bad url 5b2233d744c8b8d46c8577b69ba82a8f nan
bad url 3b713377bb9025c1ab6a184d788c940b nan
bad url 28126ac3ed45c0f1f282baa0c04232cd nan
bad url f0e1f501ec5f67e5ce9486a59304b4ab nan
bad url 267e230db9a29ddc9fb4c3b7a394a055 nan
bad url 2ed97fd85d7c7f94523d60fa1e3c3dfc nan
bad url 47e741a7840c2aa33f7c460fdad2fce2 nan
bad url 2851ef5f2ee2e0035cf316eeaeac217b nan
bad url aab39ed80019fefdabd07e7fa1125bc9 nan
bad url bb4fecc4701823a02f9c06f58e0da001 nan
bad url 0f119ca91147595c6299c830803cfb47 nan
bad url 04379559d6f119c6028c6790c2b3af1c nan
bad url ff

In [104]:
!ls ../data/fitzpatrick17k/images/ | wc -l

16527


In [105]:
small = []
issue = []
from PIL import Image
i = 0
for image_path in image_dir.glob('[!.]*'):
    try:
        img = Image.open(image_path)
        w, h = img.size
        pixels = np.array(img).sum()
        if w < 128 or h < 128 or pixels < 100_000:
            print(image_path.name, end='\t')
            print(img.size, end='\t')
            print(pixels)
            small.append(image_path)
    except Exception as e:
        print(image_path, e)
        issue.append(image_path)

68ab661c67a4b66b621eb0845f0fcccf.jpg	(130, 93)	4598402
7314c0c02f1d521c6b2b4ba955d369e8.jpg	(130, 99)	4881585
23fc2126bc1f8799fccfb195d2af6141.jpg	(133, 100)	5195807
23c779026faa7226e892c9048f43bf36.jpg	(117, 130)	3511061
c705f348b1035b9ad3d3fc2ee92ee768.jpg	(99, 130)	6332482
f968e591e15f47b544e551bc3cc5b8d3.jpg	(130, 120)	4506408
a7ef35e99387ff1227baced72467dc1f.jpg	(130, 85)	5478529
9d85c442ec9293c04423c8938cb36cad.jpg	(102, 130)	5413405
48700506ed2b3d5b0e5b4cbc6a12ad94.jpg	(130, 121)	3955616
dfae71d76e84de3d240a296bd40ff341.jpg	(130, 86)	5459411
8c4ed20439baa7bf0053b3abd8c481ee.jpg	(130, 89)	2971562
7da091d784090919f87bb8d5a1170830.jpg	(130, 120)	4926600
1dd06f10a962779a01ccae641e1c7e42.jpg	(130, 74)	2718669
742c83b0b5b1a14e8cddbef28108ae17.jpg	(130, 89)	5026602
4157d3f95f44ccc755f79cda16722730.jpg	(130, 74)	5377141
6bb966643b6a6f29047114b667aca8ec.jpg	(130, 117)	6781216
d04150a06562fb2c21073a745b42f7dc.jpg	(127, 192)	10912662
6bdffc50c482e5f7ecf1b1b325f6a172.jpg	(130, 108)	5524151


In [106]:
issue

[]

In [94]:
for p in issue:
    p.unlink()

In [17]:
import pandas as pd
df = pd.read_csv('fitzpatrick17k/fitzpatrick17k.csv')
missing = open('skin/missing.txt', 'r').read().split('\n')
df['exclude'] = df.apply(lambda r: True if r.md5hash in missing else False, axis=1)
df = df[~df.exclude]
print('pre', df.shape)

df['aggregated_fitzpatrick_scale'] = df.apply(lambda r: r.fitzpatrick_centaur if r.fitzpatrick_scale == -1 and r.fitzpatrick_centaur != -1 else r.fitzpatrick_scale, axis=1)
df['aggregated_fitzpatrick_scale'] = df.apply(lambda r: (r.aggregated_fitzpatrick_scale + r.fitzpatrick_centaur) / 2 if r.aggregated_fitzpatrick_scale != -1 and r.fitzpatrick_centaur != -1 else r.aggregated_fitzpatrick_scale, axis=1)
df['aggregated_fitzpatrick_scale'] = df.aggregated_fitzpatrick_scale.round()

labels = sorted(df.label.unique())
mapping = dict(zip(labels, range(len(labels))))
df['target'] = df.label.map(mapping)

num_test_per_scale = 200
num_val_per_scale = 100
scales = sorted([x for x in df.aggregated_fitzpatrick_scale.unique() if x != -1])
test_df = pd.concat([df[df['aggregated_fitzpatrick_scale'] == scale].sample(n=num_test_per_scale, random_state=0) for scale in scales])
_df = df[~df.md5hash.isin(test_df.md5hash)]
val_df = pd.concat([_df[_df['aggregated_fitzpatrick_scale'] == scale].sample(n=num_val_per_scale, random_state=0) for scale in scales])
train_df = df[~df.md5hash.isin(pd.concat([val_df, test_df]).md5hash)]
print(*map(len, (train_df, val_df, test_df)))

train_df['split'] = 'train'
val_df['split'] = 'val'
test_df['split'] = 'test'
df = pd.concat([train_df, val_df, test_df])
df.loc[df.aggregated_fitzpatrick_scale == -1, 'split'] = 'test'
print('post', df.shape)
print(f'{len(train_df)=}')
print(f'{len(val_df)=}')
print(f'{len(test_df)=}')
print(train_df.aggregated_fitzpatrick_scale.value_counts())
print(df.split.value_counts())

pre (16527, 10)
14727 600 1200
post (16527, 13)
len(train_df)=14727
len(val_df)=600
len(test_df)=1200
 2.0    6305
 4.0    3219
 1.0    2370
 3.0    1613
 5.0     498
 6.0     433
-1.0     289
Name: aggregated_fitzpatrick_scale, dtype: int64
train    14438
test      1489
val        600
Name: split, dtype: int64


/var/folders/39/jd0hlp_j55d4kr11j2_nj2d00000gn/T/ipykernel_37358/783053325.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['split'] = 'train'


In [63]:
from pathlib import Path
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler
from torchvision import transforms

class SkinDataset(Dataset):
    def __init__(self, image_dir, label_mapping, transform):
        super().__init__()
        self.image_dir = Path(image_dir)
        self.images = list(label_mapping.keys())
        self.label_mapping = label_mapping
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        image_path = self.image_dir / self.images[index]
        image = Image.open(image_path.with_suffix('.jpg'))
        image = self.transform(image)
        target = self.label_mapping[image_path.stem]
        # target = torch.tensor(target, dtype=torch.long)
        return image, target

train_transform=transforms.Compose([
    # transforms.ToPILImage(),
    transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
    transforms.RandomRotation(degrees=15),
    transforms.ColorJitter(),
    transforms.RandomHorizontalFlip(),
    transforms.CenterCrop(size=224),  # Image net standards
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transforms=transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(size=256),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [102]:
image_subset = [p.stem for p in Path('skin/images/').iterdir()]

In [41]:
image_df = df[df.md5hash.isin([x.stem for x in image_paths])]

fitz_1 = image_df[image_df.aggregated_fitzpatrick_scale == 1]


type(transforms.ToTensor())

In [132]:
fitz_1_train_df = df.query('aggregated_fitzpatrick_scale == 1 and split == "train" and md5hash in @image_subset')
label_count = fitz_1_train_df['target'].value_counts().sort_index()
weight_mapping = dict(1 / label_count)

sampler = WeightedRandomSampler(
    [weight_mapping[row.target] for _, row in fitz_1_train_df.iterrows()], 
    len(fitz_1_train_df),
    replacement=True,
)

fitz_1_label_mapping = dict(fitz_1_train_df[['md5hash', 'target']].values)
ds = SkinDataset('skin/images', fitz_1_label_mapping, train_transform)
dl = DataLoader(
    ds,
    batch_size=4,
    sampler=sampler,
    shuffle=False,
    # num_workers=num_workers
)

In [133]:

    [weight_mapping[row.target] for _, row in fitz_1_train_df.iterrows()], 

([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],)

In [134]:
df.aggregated_fitzpatrick_scale.unique()

[-1.0, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0]

In [122]:
next(iter(dl))[1]

tensor([17, 10, 17, 51])

In [62]:
class_sample_count = np.array(train_df[label].value_counts().sort_index())

In [62]:
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in train[label]])
samples_weight = torch.from_numpy(samples_weight)
sampler = WeightedRandomSampler(
    samples_weight.type('torch.DoubleTensor'),
    len(samples_weight),
    replacement=True)

In [142]:
df.to_csv('fitzpatrick.csv')

In [143]:
df.aggregated_fitzpatrick_scale.value_counts()

 1.5    2681
 1.0    2670
 2.0    2451
 3.0    1913
 2.5    1473
 3.5    1424
 4.0    1288
 4.5     807
 5.0     798
 6.0     367
 5.5     366
-1.0     289
Name: aggregated_fitzpatrick_scale, dtype: int64

In [148]:
Path('~/asdf/').expanduser()

PosixPath('/Users/clu/asdf')

In [149]:
Path.home()

PosixPath('/Users/clu')

In [153]:
Path('/asdf/qwer/gji').suffix

''

In [162]:
x = torch.rand(1, 224, 224)

In [163]:
x.repeat(3 if x.shape[0] == 1 else 1, 1, 1).shape

torch.Size([3, 224, 224])

In [169]:
sorted(df.target.unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113]

In [173]:
from tqdm import tqdm
for x, y in tqdm(ds):
    print(x.unsqueeze(0).shape)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 131.58it/s]

torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])


In [178]:
df.sample?

Signature:
df.sample(
    n: 'int | None' = None,
    frac: 'float | None' = None,
    replace: 'bool_t' = False,
    weights=None,
    random_state: 'RandomState | None' = None,
    axis: 'Axis | None' = None,
    ignore_index: 'bool_t' = False,
) -> 'NDFrameT'
Docstring:
Return a random sample of items from an axis of object.

You can use `random_state` for reproducibility.

Parameters
----------
n : int, optional
    Number of items from axis to return. Cannot be used with `frac`.
    Default = 1 if `frac` = None.
frac : float, optional
    Fraction of axis items to return. Cannot be used with `n`.
replace : bool, default False
    Allow or disallow sampling of the same row more than once.
weights : str or ndarray-like, optional
    Default 'None' results in equal probability weighting.
    If passed a Series, will align with target object on index. Index
    values in weights not found in sampled object will be ignored and
    index values in sampled object not in weights will be a